In [1]:
import sys
sys.path.append("../../")
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

In [2]:
from cellmating.tracking._hausdorf_tracker import Tracker
from cellmating.visualize import animate_images
from cellmating.utils import file_traverse
from cellmating.io import imread, imsave
from cellmating.utils import crop_single_object
from cellmating.visualize import label2rgb

%load_ext autoreload
%autoreload 2

In [3]:
file_root = "/media/wlli/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/"
# file_root = "/Volumes/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/"
file_list = file_traverse(file_root, file_regular=".*._rigid.tif$")

In [155]:
filename = file_list[19]
print(filename)
image = imread(filename)
image = np.moveaxis(image, -1, 1)
print(image.shape)

/media/wlli/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/YAV2104xYAV2105_005_20_rigid.tif
(241, 4, 1020, 1020)


In [156]:
trace = Tracker(image[:, -1], max_miss=10, min_hist=1, low_ratio=0.5, high_ratio=1, is_acc=False)
_ = trace()
traced_image = trace.to_image()

frame= 2 [] [1223]
divison: [[2 1 2]]
fusion: [[0 1 0]]
frame= 3 [] [1226]
divison: [[0 0 1]]
frame= 5 [] [1214]
frame= 6 [3003] []
frame= 7 [] [1217]
frame= 8 [3002] [1218]
frame= 9 [1226] [1215, 4002]
frame= 10 [3001] []
frame= 11 [] [1217, 1228]
frame= 12 [1226] [1224]
frame= 13 [1226] [2001]
divison: [[0 0 1]
 [1 2 3]]
divison: [[0 1 2]]
frame= 15 [] [1018]
divison: [[0 1 4]
 [1 3 6]
 [2 2 5]
 [3 7 8]]
frame= 16 [] [1017]
frame= 17 [1232] [1018, 1223, 1232, 1234]
divison: [[0 1 2]]
frame= 18 [] [1018]
frame= 19 [] [1017]
divison: [[0 0 2]
 [1 3 4]]
frame= 20 [] [1018]
frame= 21 [] [1018]
frame= 22 [] [1018, 1227]
divison: [[0 1 2]]
frame= 23 [1237] [1017, 1227]
divison: [[0 1 4]
 [1 2 3]]
frame= 24 [1237] [1018, 1237, 1239]
divison: [[0 1 2]]
frame= 25 [3001] [1017]
divison: [[0 1 2]]
frame= 26 [3001] [1018, 1232]
divison: [[0 3 4]
 [1 5 6]
 [2 1 2]]
frame= 27 [3001] [1017, 1237]
divison: [[0 1 2]]
frame= 28 [3001] [1017, 1233]
divison: [[0 1 2]
 [1 3 4]]
frame= 29 [3001] [1016, 12

In [157]:
imsave(filename[:-4]+"_trackv1.tif", traced_image, dtype=np.uint16, imagej=True)

In [122]:
# annotation = label2rgb(traced_image)

In [158]:
# ani = animate_images(image[200:, -1,:,:])
# ani2 = animate_images(annotation[200:])

# html1 = HTML(ani.to_jshtml()).data
# html2 = HTML(ani2.to_jshtml()).data

# # Display the combined animations
# HTML(f"""
# <div style="display: flex;">
#     <div style="flex: 1; margin-right: 10px;">
#         {html1}
#     </div>
#     <div style="flex: 1; margin-left: 10px;">
#         {html2}
#     </div>
# </div>
# """)

In [159]:
from cellmating.utils import crop_object_to_dict
import json
from cellmating.utils import dump_to_pkl
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [160]:
labels = np.unique(traced_image[-1])[1:]
labels = list(labels[labels>2000])

In [148]:
labels = [i for i in labels if i not in [3431, 3400]]

In [161]:
labels

[3338,
 3368,
 3371,
 3372,
 3375,
 3381,
 3387,
 3392,
 3393,
 3401,
 3405,
 3410,
 3419,
 3421,
 3429,
 3430,
 3431,
 3448,
 4382,
 4396,
 4440]

In [150]:
roi_dict = crop_object_to_dict(traced_image[-1], length=250, labels=labels)
with open(filename[:-4]+'.json', 'w') as json_file:
    json.dump(roi_dict, json_file, indent=4)

In [151]:
traced_image_stack = np.concatenate((image, traced_image[:,None,:,:]), axis=1)
imsave(filename[:-4]+"_track.tif", traced_image_stack, dtype=np.uint16, imagej=True)

In [152]:
dump_to_pkl(trace, filename[:-4]+".tracker")

In [213]:
# with open('data.json', 'r') as json_file:
#     loaded_data = json.load(json_file)
# print(loaded_data)